周内效应——定投篇

In [1]:
import pandas as pd
from datetime import datetime
import trdb2py
import numpy as np

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [12]:
# 具体基金
asset = 'jqdata.000300_XSHG|1d'
# baselineasset = 'jrj.510310'
# baselineasset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy2 = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=0.5,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

# 卖出参数，全部卖出
paramssell7 = trdb2py.trading2_pb2.SellParams(
#     perVolume=1,
    keepTime=7 * 24 * 60 * 60,
)

lststart = [1, 2, 3, 4, 5]
lsttitle = ['周一', '周二', '周三', '周四', '周五']

In [3]:
def calcweekday2val2(wday, offday):
    if offday == 1:
        if wday == 5:
            return 3
    if offday == 2:
        if wday >= 4:
            return 4
    if offday == 3:
        if wday >= 3:
            return 5
    if offday == 4:
        if wday >= 2:
            return 6
        
    return offday


大家好，我是格子衫小C，周内效应原计划是3篇，现在准备调整一下节奏，接下来更新频次会高一些，相对每篇内容也会少一点。

前面主要在分析周内效应是否存在，在不同和的市场和不同的标的之间横向比较，基本可以得出结论——A股市场，周4跌的概率要稍高，在29日均线以下，周1的跌幅较高。这个规律，在A股的大多数指数里都有效，而且中盘最为明显，小盘其次，大盘最次。

接下来我们看看可以如何来利用这个结论。

首先，既然是周4跌的概率高一些，最先想到的应该就是周4定投买入了，咱们看看这样的策略怎样。

In [13]:
# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

In [22]:
lstparams = []

for i in range(0, 5):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[lststart[i]],
    )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )
    
    paramsaip = trdb2py.trading2_pb2.AIPParams(
        money=10000,
        type=trdb2py.trading2_pb2.AIPTT_WEEKDAY,
        day=lststart[i],
    )

    s0.buy.extend([buy0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
#     s0.paramsInit.CopyFrom(paramsinit)
    s0.paramsAIP.CopyFrom(paramsaip)
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='{}定投'.format(lsttitle[i]),
    ))
 
lstaippnl = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstaippnl + [pnlBaseline], toImg=isStaticImg, width=width, height=height)